<a href="https://colab.research.google.com/github/LikhithaKANURU/MOVIE-RECOMMENDATION-SYSTEM/blob/main/MOBILE_RECOMMENDATION_SYSTEM_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **MOBILE RECOMMENDATION SYSTEM**

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'movie-recommendation-system:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3375918%2F5872805%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240731%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240731T163058Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4e82a775e1641b8e410adfcecdfb0dd4247e21c72db77fded32cbd32eec1c11afc448156a8edd65576f146e3f19f5d917a13666d2b2b6313e6fc19da215f9e9f8492f3ad299d2f712db50e83d708e8b906b829f8ad00b09568531fc48385a595a07b608248399bf2fd1c7fd5d3227562ea851fb928d64147978b2b5fca073b613cb8eae057b01d783baa58f580a32785b9badb056bcef39a65958e93a68a5880a2b9a6e15fb22bb60fa1bb6bf6fd1519bb40129659c19f79ab3b6ea53441e506ff2987907c40da4f4c766f821e27559f787e62c0150d3e54d665da97020d675449cbafd8f9b670e8c8dc3fafbb98ae122666213ecd284fedbdaa5d14b508d878'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 172958815 bytes downloaded
Downloaded and uncompressed: movie-recommendation-system
Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movie-recommendation-system/ratings.csv
/kaggle/input/movie-recommendation-system/movies.csv


# Movie Recommendation using KNN

In [3]:
movies_df=pd.read_csv('/kaggle/input/movie-recommendation-system/movies.csv')
ratings_df=pd.read_csv('/kaggle/input/movie-recommendation-system/ratings.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
movies_df.shape

(62423, 3)

In [5]:
movies_df.isna().sum()

,0
movieId,0
title,0
genres,0


In [6]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  62423 non-null  int64 
 1   title    62423 non-null  object
 2   genres   62423 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [7]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [8]:
ratings_df.isna().sum()

,0
userId,0
movieId,0
rating,0
timestamp,0


In [9]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000095 entries, 0 to 25000094
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 762.9 MB


In [10]:
ratings_df.shape

(25000095, 4)

In [11]:
df=movies_df.merge(ratings_df)
df1=df.drop(['genres', 'timestamp'], axis=1)
df1.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),2,3.5
1,1,Toy Story (1995),3,4.0
2,1,Toy Story (1995),4,3.0
3,1,Toy Story (1995),5,4.0
4,1,Toy Story (1995),8,4.0


In [12]:
df2=df1.groupby('title')['rating'].count().reset_index().rename(columns={'rating': 'ratingcount'})
df2.head()

,title,ratingcount
0,"""BLOW THE NIGHT!"" Let's Spend the Night Togeth...",1
1,"""Great Performances"" Cats (1998)",179
2,#1 Cheerleader Camp (2010),9
3,#Captured (2017),2
4,#Female Pleasure (2018),3


In [13]:
df3=df1.merge(df2,on='title',how='left')
df3.head()

,movieId,title,userId,rating,ratingcount
0,1,Toy Story (1995),2,3.5,57309
1,1,Toy Story (1995),3,4.0,57309
2,1,Toy Story (1995),4,3.0,57309
3,1,Toy Story (1995),5,4.0,57309
4,1,Toy Story (1995),8,4.0,57309


In [14]:
print(f'10% of count= {df3.ratingcount.quantile(.1)}')
print(f'25% of count= {df3.ratingcount.quantile(.25)}')
print(f'50% of count= {df3.ratingcount.quantile(.5)}')
print(f'75% of count= {df3.ratingcount.quantile(.75)}')
print(f'85% of count= {df3.ratingcount.quantile(.85)}')
print(f'100% of count= {df3.ratingcount.quantile(1)}')

10% of count= 825.0
25% of count= 2986.0
50% of count= 9152.0
75% of count= 20757.0
85% of count= 30910.0
100% of count= 81491.0


In [15]:
df4=df3[df3.ratingcount>=10000]
df4.shape

(11878413, 5)

In [16]:
movie_rating_pivote=df4.pivot_table(index='title',columns='userId', values='rating').fillna(0)
movie_rating_pivote.head()

userId,1,2,3,4,5,6,7,8,9,10,...,162532,162533,162534,162535,162536,162537,162538,162539,162540,162541
title,,,,,,,,,,,,,,,,,,,,,
(500) Days of Summer (2009),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,2.5,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0
12 Angry Men (1957),0.0,0.0,0.0,0.0,0.0,5.0,0.0,4.0,0.0,0.0,...,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),0.0,0.0,5.0,4.0,0.0,4.0,0.0,5.0,3.0,4.5,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
28 Days Later (2002),0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,...,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

movie_rating_df=csr_matrix(movie_rating_pivote.values)

model_knn=NearestNeighbors(metric='cosine', algorithm='brute',n_neighbors=11)
model_knn.fit(movie_rating_df)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=11)

In [18]:
movie_rating_df.shape

(588, 162109)

In [19]:
distance,indices=model_knn.kneighbors(movie_rating_pivote.iloc[np.random.choice(movie_rating_df.shape[0]),:].values.reshape(1,-1),n_neighbors=11)

for i in range(0, len(distance.flatten())):
    if i == 0:
        print(f'Recommendation for a movie: {movie_rating_pivote.index[indices.flatten()[i]]}')
    else:
        print(f'{i}: {movie_rating_pivote.index[indices.flatten()[i]]} with a distance {distance.flatten()[i]}')

Recommendation for a movie: 40-Year-Old Virgin, The (2005)
1: Wedding Crashers (2005) with a distance 0.4552277713942262
2: Superbad (2007) with a distance 0.4909998103194493
3: Borat: Cultural Learnings of America for Make Benefit Glorious Nation of Kazakhstan (2006) with a distance 0.5384954860901732
4: Hangover, The (2009) with a distance 0.5603841878458953
5: School of Rock (2003) with a distance 0.5716144903022121
6: Bruce Almighty (2003) with a distance 0.5863622062027709
7: Batman Begins (2005) with a distance 0.5873638519096219
8: 300 (2007) with a distance 0.588299552254754
9: Napoleon Dynamite (2004) with a distance 0.588403448520157
10: Meet the Parents (2000) with a distance 0.5897558654362681
